In [18]:
import requests
import pandas as pd
import hopsworks
import datetime
import matplotlib.pyplot as plt
import json
import re
import os
import sys
from dotenv import load_dotenv
from datetime import datetime, timedelta, timezone
import warnings
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
import util
sys.path.append(os.path.abspath(os.path.join('..', 'model')))
import nn
import torch
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split


warnings.filterwarnings("ignore")

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 

2024-12-31 13:29:33,293 INFO: Initializing external client
2024-12-31 13:29:33,294 INFO: Base URL: https://c.app.hopsworks.ai:443

Multiple projects found. 

	 (1) KTH_ID2223
	 (2) simon003
Invalid input, should be an integer from the list of projects.
2024-12-31 13:29:45,820 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164449


In [4]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="price_prediction_model",
    version=1,
)

saved_model_dir = retrieved_model.download()

In [5]:
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score='6.23309E-1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [23]:
today = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
yesterday = today - timedelta(days=12)

In [26]:
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
power_fg = fs.get_feature_group(
    name='power',
    version=1,
)

weather_batch_data = (
    weather_fg.filter(weather_fg.date >= yesterday)
    .filter(weather_fg.date < today)
    .read()
)

power_batch_data = (
    power_fg.filter(power_fg.date >= yesterday)
    .filter(power_fg.date < today)
    .read()
)


batch_data = weather_batch_data.merge(power_batch_data, on="date", how="inner")
print(batch_data)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.81s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.49s) 
                       date  temperature_2m_mean  precipitation_sum  \
0 2024-12-20 00:00:00+00:00                 0.14                0.2   

   wind_speed_10m_max  wind_direction_10m_dominant  sunshine_duration  \
0           17.748037                   283.297974                0.0   

      hydro_mw   nuclear_mw  other_mw      wind_mw  
0  8964.833333  5823.208333   635.375  7656.208333  


In [27]:
batch_data['el_prices'] = retrieved_xgboost_model.predict(
    batch_data[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant', 'sunshine_duration', 'hydro_mw', 'nuclear_mw', 'other_mw', 'wind_mw']])
batch_data

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,sunshine_duration,hydro_mw,nuclear_mw,other_mw,wind_mw,el_prices
0,2024-12-20 00:00:00+00:00,0.14,0.2,17.748037,283.297974,0.0,8964.833333,5823.208333,635.375,7656.208333,0.555752
